In [ ]:
import pandas as pd
import youtube_dl
from moviepy.editor import *
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip, ffmpeg_resize
import boto3
import glob
from torchvision.io import read_video
from moviepy.audio import fx
import signal

In [ ]:
s3 = boto3.resource('s3')

In [ ]:
df = pd.read_csv("../unbalanced_train_segments.csv", header=[0,1,2]).iloc[400000:500000,:]

In [ ]:
df

In [ ]:
df.columns = ["link", "start_second", "end_second", "labels"]

In [ ]:
len(df)

In [ ]:
with open("list_downloaded_videos5.txt", 'r') as f:
    list_links = []
    for line in f.readlines():
        list_links.append(line.replace("\n", ""))

In [ ]:
historic = pd.read_csv("list_downloaded_videos5.txt", header=None)

In [ ]:
list_links = list(historic[0])

In [ ]:
df = df[~df['link'].isin(list_links)]

In [ ]:
len(df)

In [ ]:
df = df[df["end_second"]-df["start_second"]>=10000]

In [ ]:
len(df)

In [ ]:
def signal_handler(signum, frame):
    raise Exception("Timed out!")

In [ ]:
N_VIDEO_FRAMES = 150
N_AUDIO_FRAMES = 80000

In [ ]:
for idx, row in df.iterrows():
    complete_link = 'https://www.youtube.com/watch?v=%s'%row["link"]
    Path = "tmp5/"
    Location = '%s %(title)s.%(ext)s'.replace("%s ", Path)
    ytdl_format_options = {
    'outtmpl': Location
    }
    try:
        with youtube_dl.YoutubeDL(ytdl_format_options) as ydl:
            info_dict = ydl.extract_info(complete_link, download=False)
            title = info_dict.get('title', None)
            duration = info_dict.get('duration', None)
            signal.signal(signal.SIGALRM, signal_handler)
            signal.alarm(20) 
            ydl.download([complete_link])
            ffmpeg_extract_subclip(Path+title+".mp4", row["start_second"]/1000, (row["start_second"]+5000)/1000, targetname=Path+"cut.mp4")
            clip = VideoFileClip(Path+"cut.mp4").fx(afx.audio_normalize)
            clip = clip.resize(newsize=(224,224))
            clip.write_videofile(Path+"resize.mp4", audio_fps=16000, verbose=0, ffmpeg_params=['-ac', '1'], fps=30)
            clip = VideoFileClip(Path+"resize.mp4")
            video = read_video(Path+"resize.mp4")
            if video[1].shape[1]>=N_AUDIO_FRAMES and video[0].shape[0]>=N_VIDEO_FRAMES:
                s3.meta.client.upload_file(Path+"resize.mp4", 'tfm-historico-videos', "videos/"+title+".mp4")
            files = glob.glob(Path+'*')
            for f in files:
                os.remove(f)
            with open("list_downloaded_videos5.txt", 'a') as f:
                f.write(row["link"]+","+"SI"+"\n")
    except Exception as err:
        print(err)
        with open("list_downloaded_videos5.txt", 'a') as f:
                f.write(row["link"]+","+"NO"+"\n")
        continue

In [ ]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('tfm-historico-videos')

In [ ]:
from torchvision.io import read_video
for object in bucket.objects.all():
    bucket.download_file(object.key, "/tmp/video.mp4")
    s3.Object('tfm-historico-videos', object.key).delete()